# Logic trees for Nath & Thingbaijam (2012)

Read logic tree tables extracted from publication, along with a table of supplementary information write them to TEX and XML.

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd

%autoreload 2
import logic_tree_tools as ltt

from openquake.commonlib import nrml

## GMPE Logic Tree

In [3]:
gsim_tree_tsv = 'gmpe_logic_tree.tsv'
gsim_tree_deprecated_xml = gsim_tree_tsv.replace('.tsv','_deprecated.xml')
gsim_tree_xml = gsim_tree_tsv.replace('.tsv','.xml')

In [4]:
gsim_tree_df = ltt.read_tree_tsv(gsim_tree_tsv)

In [5]:
gsim_tree_df.drop(['Tectonic Region Type', 'Qualifier', 'Short Names'], axis=1)

,uncertaintyType,applyToTectonicRegionType,uncertaintyModel
0,gmpeModel,active shallow crust normal,"[AkkarBommer2010, BooreAtkinson2008, CampbellB..."
1,gmpeModel,active shallow crust strike-slip reverse,"[AkkarBommer2010, BooreAtkinson2008, CampbellB..."
2,gmpeModel,intraplate margin lower,"[AtkinsonBoore2006, ToroEtAl2002, SharmaEtAl20..."
3,gmpeModel,intraplate margin upper,"[AtkinsonBoore2006, ToroEtAl2002, SharmaEtAl20..."
4,gmpeModel,stable shallow crust,"[AtkinsonBoore2006, ToroEtAl2002, Campbell2003..."
5,gmpeModel,subduction interface,"[AtkinsonBoore2003SInter, ZhaoEtAl2006SInter, ..."
6,gmpeModel,subduction interface megathrust,"[AtkinsonBoore2003SInter, AtkinsonMacias2009, ..."
7,gmpeModel,subduction intraslab Himalayas,"[AtkinsonBoore2003SSlabJapan, YoungsEtAl1997SS..."
8,gmpeModel,subduction intraslab,"[AtkinsonBoore2003SSlabCascadia, YoungsEtAl199..."


In [6]:
ltt.write_gsim_tree_nrml(gsim_tree_df, gsim_tree_deprecated_xml, validate=True)

Level 1 model 5 "SharmaEtAl2009" not valid GSIM. Omitting ...
Level 1 model 4 "KannoEtAl2006Shallow" not valid GSIM. Omitting ...
Level 2 model 4 "KannoEtAl2006Shallow" not valid GSIM. Omitting ...
Level 3 model 4 "NathEtAl2012Lower" not valid GSIM. Omitting ...
Level 3 model 3 "SharmaEtAl2009" not valid GSIM. Omitting ...
Level 4 model 4 "NathEtAl2012Upper" not valid GSIM. Omitting ...
Level 4 model 3 "SharmaEtAl2009" not valid GSIM. Omitting ...
Level 5 model 4 "RagukanthIyengar2007" not valid GSIM. Omitting ...
Level 6 model 3 "KannoEtAl2006Deep" not valid GSIM. Omitting ...
Level 7 model 4 "KannoEtAl2006Deep" not valid GSIM. Omitting ...
Level 8 model 1 "AtkinsonBoore2003SSlabJapan" not valid GSIM. Omitting ...
Level 9 model 4 "Gupta2010SSlab" not valid GSIM. Omitting ...
Level 9 model 1 "AtkinsonBoore2003SSlabCascadia" not valid GSIM. Omitting ...


<string>:2: DeprecationWarning: logic_tree_tools.write_gsim_tree_nrml has been deprecated. Use gsim_data_to_tree instead


In [7]:
gsim_tree_deprecated = nrml.read(gsim_tree_deprecated_xml)

In [8]:
gsim_tree = ltt.df_to_tree(gsim_tree_df, validate=True)
with open(gsim_tree_xml, 'w+') as f:
    nrml.write([gsim_tree], f, fmt='%g')

bs1 model 5 "SharmaEtAl2009" not valid GSIM. Omitting ...
bs1 model 4 "KannoEtAl2006Shallow" not valid GSIM. Omitting ...
bs2 model 4 "KannoEtAl2006Shallow" not valid GSIM. Omitting ...
bs3 model 4 "NathEtAl2012Lower" not valid GSIM. Omitting ...
bs3 model 3 "SharmaEtAl2009" not valid GSIM. Omitting ...
bs4 model 4 "NathEtAl2012Upper" not valid GSIM. Omitting ...
bs4 model 3 "SharmaEtAl2009" not valid GSIM. Omitting ...
bs5 model 4 "RagukanthIyengar2007" not valid GSIM. Omitting ...
bs6 model 3 "KannoEtAl2006Deep" not valid GSIM. Omitting ...
bs7 model 4 "KannoEtAl2006Deep" not valid GSIM. Omitting ...
bs8 model 1 "AtkinsonBoore2003SSlabJapan" not valid GSIM. Omitting ...
bs9 model 4 "Gupta2010SSlab" not valid GSIM. Omitting ...
bs9 model 1 "AtkinsonBoore2003SSlabCascadia" not valid GSIM. Omitting ...


In [9]:
gsim_tree_read = nrml.read(gsim_tree_xml)

In [10]:
# not clear why you don't get the same thing reading and writing
print gsim_tree.logicTreeBranchingLevel.logicTreeBranchSet.logicTreeBranch.uncertaintyModel.text
print gsim_tree_read.logicTree.logicTreeBranchingLevel.logicTreeBranchSet.logicTreeBranch.uncertaintyModel.text.strip()
print gsim_tree_deprecated.logicTree.logicTreeBranchingLevel.logicTreeBranchSet.logicTreeBranch.uncertaintyModel.text.strip()

AkkarBommer2010
AkkarBommer2010
AkkarBommer2010


In [11]:
node = gsim_tree[0][0]
print node
print '%s = %dx %s' %(ltt.strip_fqtag(node.tag), len(node), 
                      ltt.strip_fqtag(node[0].tag))
print ltt.get_dict_key_match(node.attrib,'id')

<logicTreeBranchSet {'branchSetID': 'bs1', 'applyToTectonicRegionType': 'active shallow crust normal', 'uncertaintyType': 'gmpeModel'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID


In [12]:
ltt.nrml_to_pdf(gsim_tree_xml, include_ids=False)

In [13]:
# let's also write an un-validated ("full") version of the tree
gsim_tree_all = ltt.df_to_tree(gsim_tree_df, validate=False)
gsim_tree_all_xml = gsim_tree_xml.replace('.xml','_all.xml')
with open(gsim_tree_all_xml, 'w+') as f:
    nrml.write([gsim_tree_all], f, fmt='%g')
ltt.nrml_to_pdf(gsim_tree_all_xml, include_ids=False)

## Source Model Logic Tree

In [14]:
ltt.nrml_to_pdf('example_source_model_logic_tree.xml', include_ids=False)
ltt.nrml_to_pdf('example_gmpe_logic_tree.xml', include_ids=False)

Too many (11) nodes in logicTree, abbreviating to first & last 5


In [15]:
source_tree_tsv = 'source_model_logic_tree.tsv'
source_tree_xml = source_tree_tsv.replace('.tsv','.xml')
source_tree_simplified_xml = source_tree_tsv.replace('.tsv','_simplified.xml')

In [16]:
source_tree_df = ltt.read_tree_tsv(source_tree_tsv)
source_tree_df

,uncertaintyType,applyToSources,uncertaintyModel,uncertaintyWeight
0,sourceModel,all,"[areal_source_model.xml, smoothed_source_model...","[0.4, 0.27, 0.33]"
1,maxMagGRRelative,areal_source_model.tsv,"[-stdmmax, 0, +stdmmax]","[0.32, 0.36, 0.32]"
2,bGRRelative,areal_source_model.tsv,"[-stdb, 0, +stdb]","[0.32, 0.36, 0.32]"


In [17]:
source_tree_simplified = ltt.df_to_tree(source_tree_df, validate=True)
with open(source_tree_simplified_xml, 'w+') as f:
    nrml.write([source_tree_simplified], f, fmt='%g')

In [18]:
node = source_tree_simplified[0][0]
print node
print '%s = %dx %s' %(ltt.strip_fqtag(node.tag), len(node), 
                      ltt.strip_fqtag(node[0].tag))
print ltt.get_dict_key_match(node.attrib,'id')
print node[0].uncertaintyModel

<logicTreeBranchSet {'branchSetID': 'bs1', 'uncertaintyType': 'sourceModel'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID
<uncertaintyModel {} areal_source_model.xml >


In [19]:
ltt.nrml_to_pdf(source_tree_simplified_xml, include_ids=False)

In [20]:
source_tree_expanded_df = ltt.df_expand_sources(source_tree_df)
pd.concat((source_tree_expanded_df.head(), source_tree_expanded_df.tail()))

Cannot find 0 in keys of areal_source_model.tsv, setting to zero
Cannot find 0 in keys of areal_source_model.tsv, setting to zero


,uncertaintyType,applyToSources,uncertaintyModel,uncertaintyWeight
0,sourceModel,all,"[areal_source_model.xml, smoothed_source_model...","[0.4, 0.27, 0.33]"
1,maxMagGRRelative,1,"[-0.4, 0.0, 0.4]","[0.32, 0.36, 0.32]"
2,bGRRelative,1,"[-0.07, 0.0, 0.07]","[0.32, 0.36, 0.32]"
3,maxMagGRRelative,1.1,"[-0.4, 0.0, 0.4]","[0.32, 0.36, 0.32]"
4,bGRRelative,1.1,"[-0.07, 0.0, 0.07]","[0.32, 0.36, 0.32]"
256,bGRRelative,933,"[-0.13, 0.0, 0.13]","[0.32, 0.36, 0.32]"
257,maxMagGRRelative,935,"[-0.3, 0.0, 0.3]","[0.32, 0.36, 0.32]"
258,bGRRelative,935,"[-0.09, 0.0, 0.09]","[0.32, 0.36, 0.32]"
259,maxMagGRRelative,936,"[-0.2, 0.0, 0.2]","[0.32, 0.36, 0.32]"
260,bGRRelative,936,"[-0.11, 0.0, 0.11]","[0.32, 0.36, 0.32]"


In [21]:
source_tree_expanded = ltt.df_to_tree(source_tree_expanded_df, validate=False)

In [22]:
node = source_tree_expanded[2][0]
print node
print '%s = %dx %s' %(ltt.strip_fqtag(node.tag), len(node), 
                      ltt.strip_fqtag(node[0].tag))
print ltt.get_dict_key_match(node.attrib,'id')
model = node[0].uncertaintyModel
print model

<logicTreeBranchSet {'branchSetID': 'bs3', 'applyToSources': 1.0, 'uncertaintyType': 'bGRRelative'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID
<uncertaintyModel {} -0.07 >


In [23]:
with open(source_tree_xml, 'w+') as f:
    nrml.write([source_tree_expanded], f, fmt='%g')
ltt.nrml_to_pdf(source_tree_xml, include_ids=False)

Too many (261) nodes in logicTree, abbreviating to first & last 5
